# Etude de la prédiction des arrêts du coeur

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score

## Fonction de chargement des données et de prévisualisation

In [5]:
"""
Charge un fichier CSV et affiche les premières lignes du DataFrame.

Paramètres :
- chemin : Chemin vers le fichier CSV.
- n : Nombre de premières lignes à afficher (par défaut, 10).

Return :
- DataFrame contenant les données selon la valeur de n.
"""
def charger_visualiser(chemin, n=10):
    data = pd.read_csv(chemin)
    return data.head(n)

# Utilisation de la fonction
chemin = 'heart.csv'
visu = charger_visualiser(chemin)
print(visu)

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   
5   39   M           NAP        120          339          0     Normal    170   
6   45   F           ATA        130          237          0     Normal    170   
7   54   M           ATA        110          208          0     Normal    142   
8   37   M           ASY        140          207          0     Normal    130   
9   48   F           ATA        120          284          0     Normal    120   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1     

## Fonction des test d'indépendance 
Cette fonction permet de réaliser le test du chi-2 et du V de Cramer

In [9]:
"""
Effectue le test du khi-2 et calcule la valeur V de Cramer pour une table de contingence donnée.

Paramètres :
- data : DataFrame contenant les données.
- colonne_cible : Nom de la colonne cible (par exemple, 'HeartDisease').
- colonne_caract : Nom de la colonne de caractéristique pour la table de contingence (par exemple, 'ChestPainType').

Returns :
- chi2 : Statistique chi2 du test du khi-2.
- p : Valeur p du test du khi-2.
- v_cramer : Valeur V de Cramer calculée.
"""

def test_chi2_cramer(data, colonne_cible, colonne_caract):

    # Créer un tableau de contingence
    contingency_table = pd.crosstab(data[colonne_cible], data[colonne_caract])

    # Effectuer le test du khi-2
    chi2, p, _, _ = chi2_contingency(contingency_table)
    print(f"Test du khi-2 : chi2 = {chi2}, p-value = {p}")

    # Calculer la valeur V de Cramer
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape)
    v_cramer = (chi2 / n) ** 0.5 / (min_dim - 1)

    print(f"V de Cramer : {v_cramer}")

    return chi2, p, v_cramer

# Charger le fichier CSV
chemin = 'heart.csv'
data = pd.read_csv(chemin)

# Utilisation de la fonction
chi2, p, v_cramer = test_chi2_cramer(data, 'HeartDisease', 'ChestPainType')

Test du khi-2 : chi2 = 268.06723902181767, p-value = 8.08372842808765e-58
V de Cramer : 0.5403815716169053


In [13]:
def prepare_data(data, colonne_cible):
    """Préparation des données."""
    X = data.drop(colonne_cible, axis=1)
    y = data[colonne_cible]
    X = pd.get_dummies(X)
    return X, y

def divise_data(X, y, test_size=0.2, random_state=42):
    """Diviser le dataset en ensemble d'entraînement et ensemble de test."""
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def logistic_regression(X_train, y_train):
    """Créer et entraîner le modèle de régression logistique."""
    model = LogisticRegression()
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    """Évaluation du modèle."""
    y_pred = model.predict(X_test)
    print("\nÉvaluation du modèle:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    return y_pred

def plot_roc(y_test, y_prob):
    """Afficher la courbe ROC."""
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    roc_auc = roc_auc_score(y_test, y_prob)
    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
    plt.title('Courbe ROC')
    plt.xlabel('Taux de faux positifs (FPR)')
    plt.ylabel('Taux de vrais positifs (TPR)')
    plt.legend()
    plt.show()